In [ ]:
import os
import sys

os.environ["STEAM__API_KEY"] = "id"
os.environ["FABRIC__WORKSPACE_ID"] = "id"
os.environ["FABRIC__BRONZE_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__SILVER_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__GOLD_LAKEHOUSE_ID"] = "id"

sys.path.insert(0, "/lakehouse/default/Files")

from src.steam_analytics.config import Settings
from pyspark.sql import functions as F

settings = Settings()


print("=" * 60)
print("PIPELINE HEALTH CHECK")
print("=" * 60)

layers = [
    ("Bronze", settings.fabric.bronze_abfss_path, ["raw_steam_store", "raw_steam_reviews", "raw_steam_player_stats"]),
    ("Silver", settings.fabric.silver_abfss_path, ["dim_games", "dim_game_reviews", "fact_player_counts"]),
    ("Gold", settings.fabric.gold_abfss_path, ["agg_game_metrics", "agg_price_history", "agg_genre_summary"]),
]

for layer_name, path, tables in layers:
    print(f"\n{layer_name.upper()} LAYER")
    print("-" * 40)
    for table in tables:
        try:
            df = spark.read.format("delta").load(f"{path}/Tables/{table}")
            count = df.count()
            print(f"  Success {table}: {count:,} records")
        except Exception as e:
            print(f"  Failed {table}: {e}")